In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12634716710723773710
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1453034700
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3988732016876304360
physical_device_desc: "device: 0, name: GeForce GTX 750 Ti, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [26]:
mnist = tf.keras.datasets.mnist.load_data()
(x_trainx, y_train), (x_testx, y_test) = mnist

x_train = np.reshape(x_trainx,(60000,784))
x_test = np.reshape(x_testx,(10000,784))

#regulazation
x_train = x_train / 255
x_test = x_test / 255

y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]


#batch
###very important
batch_size = 50
batch_num = 60000 // batch_size
x_train = np.split(np.array(x_train),batch_num)
y_train = np.split(np.array(y_train),batch_num)



In [7]:
#build network
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

#dropout
keep_prob=tf.placeholder(tf.float32)

w1 = tf.Variable(tf.truncated_normal([784,1000],stddev=0.1))
b1 = tf.Variable(tf.zeros([1000])+0.1)
L1 = tf.nn.relu(tf.matmul(x,w1)+b1)
L1_drop = tf.nn.dropout(L1,keep_prob)

w2 = tf.Variable(tf.truncated_normal([1000,1000],stddev=0.1))
b2 = tf.Variable(tf.zeros([1000])+0.1)
L2 = tf.nn.relu(tf.matmul(L1,w2)+b2)
L2_drop = tf.nn.dropout(L2,keep_prob)

w3 = tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b3 = tf.Variable(tf.zeros([10])+0.1)
L3 = tf.matmul(L2,w3)+b3

predict = tf.nn.softmax(L3)


#loss
# loss = tf.reduce_mean(tf.square(y - predict))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=predict))


#gradient
# train_step = tf.train.GradientDescentOptimizer(0.4).minimize(loss)
train_step = tf.train.ProximalGradientDescentOptimizer(0.5).minimize(loss)
# train_step = tf.train.AdamOptimizer(1e-2).minimize(loss)

init = tf.global_variables_initializer()

#accuracy
correct = tf.equal(tf.argmax(y,1),tf.argmax(predict,1))
acc = tf.reduce_mean(tf.cast(correct,tf.float32))


#batch
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        for batch in range(batch_num):
            sess.run(train_step,feed_dict={x:x_train[batch],y:y_train[batch],keep_prob:0.7})    
        test_accuracy = sess.run(acc, feed_dict={x:x_test,y:y_test})
        train_accuracy = sess.run(acc, feed_dict={x:x_train[0],y:y_train[0],keep_prob:1.0})
        print("test:"+str(test_accuracy)+"  train:" +str(train_accuracy))

test:0.6422  train:0.74
test:0.8498  train:0.88
test:0.9578  train:0.96
test:0.9661  train:0.98
test:0.9713  train:1.0
test:0.9684  train:1.0
test:0.9779  train:1.0
test:0.9772  train:1.0
test:0.9784  train:1.0
test:0.979  train:1.0
test:0.9807  train:1.0
test:0.9819  train:1.0
test:0.9822  train:1.0
test:0.9811  train:1.0
test:0.9835  train:1.0
test:0.9819  train:1.0
test:0.9818  train:1.0
test:0.9826  train:1.0
test:0.9832  train:1.0
test:0.9831  train:1.0
test:0.9825  train:1.0


In [27]:
#build network
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
lr = tf.Variable(0.001,tf.float32)

#dropout
keep_prob=tf.placeholder(tf.float32)

w1 = tf.Variable(tf.truncated_normal([784,1000],stddev=0.1))
b1 = tf.Variable(tf.zeros([1000])+0.1)
L1 = tf.nn.relu(tf.matmul(x,w1)+b1)
L1_drop = tf.nn.dropout(L1,keep_prob)

w2 = tf.Variable(tf.truncated_normal([1000,1000],stddev=0.1))
b2 = tf.Variable(tf.zeros([1000])+0.1)
L2 = tf.nn.relu(tf.matmul(L1,w2)+b2)
L2_drop = tf.nn.dropout(L2,keep_prob)

w3 = tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b3 = tf.Variable(tf.zeros([10])+0.1)
L3 = tf.matmul(L2,w3)+b3

predict = tf.nn.softmax(L3)


#loss
# loss = tf.reduce_mean(tf.square(y - predict))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=predict))

 
#gradient
# train_step = tf.train.GradientDescentOptimizer(0.4).minimize(loss)
# train_step = tf.train.ProximalGradientDescentOptimizer(0.5).minimize(loss)
train_step = tf.train.AdamOptimizer(lr).minimize(loss)


init = tf.global_variables_initializer()

#accuracy
correct = tf.equal(tf.argmax(y,1),tf.argmax(predict,1))
acc = tf.reduce_mean(tf.cast(correct,tf.float32))


#batch
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        sess.run(tf.assign(lr,0.001 * (0.9 ** epoch)))
        for batch in range(batch_num):
            sess.run(train_step,feed_dict={x:x_train[batch],y:y_train[batch],keep_prob:0.7})
        for batch in range(batch_num):
            sess.run(train_step,feed_dict={x:x_train[batch],y:y_train[batch],keep_prob:0.5})    
        lrp = sess.run(lr)    
        test_accuracy = sess.run(acc, feed_dict={x:x_test,y:y_test})
        train_accuracy = sess.run(acc, feed_dict={x:x_train[0],y:y_train[0],keep_prob:1.0})
        print("epoch:" +str(epoch) +" test:"+str(test_accuracy)+"  train:" +str(train_accuracy) + " lr:" + str(lrp))

epoch:0 test:0.7695  train:0.82 lr:0.001
epoch:1 test:0.8567  train:0.88 lr:0.0009
epoch:2 test:0.9575  train:0.96 lr:0.00081
epoch:3 test:0.9616  train:0.98 lr:0.000729
epoch:4 test:0.9609  train:0.98 lr:0.0006561
epoch:5 test:0.9635  train:0.98 lr:0.00059049
epoch:6 test:0.9674  train:0.98 lr:0.000531441
epoch:7 test:0.9698  train:0.98 lr:0.0004782969
epoch:8 test:0.9763  train:0.98 lr:0.00043046722
epoch:9 test:0.9736  train:1.0 lr:0.0003874205
epoch:10 test:0.9762  train:1.0 lr:0.00034867844
epoch:11 test:0.9755  train:1.0 lr:0.0003138106
epoch:12 test:0.9784  train:1.0 lr:0.00028242954
epoch:13 test:0.9772  train:0.98 lr:0.00025418657
epoch:14 test:0.9782  train:1.0 lr:0.00022876792
epoch:15 test:0.9795  train:1.0 lr:0.00020589113
epoch:16 test:0.9807  train:1.0 lr:0.00018530202
epoch:17 test:0.98  train:1.0 lr:0.00016677182
epoch:18 test:0.981  train:1.0 lr:0.00015009464
epoch:19 test:0.9814  train:1.0 lr:0.00013508517
epoch:20 test:0.9823  train:1.0 lr:0.00012157665
